# Extract subset
Do wykonania tego kroku potrzebny jest plink.

In [3]:
import subprocess
import pandas as pd

dataDir = 'data'
inputPedMap = 'data/HACK_15_one'

assoc = pd.read_csv('data/Marta-ped_sorted.assoc', sep=' ')

extractNums = [250, 500, 750, 1000]
# extractNums = [70, 100, 200, 250, 500, 750, 1000]

for extractNum in extractNums:

    snpsPath = dataDir+'/SNPs'+str(extractNum)+'.txt'

    assoc['SNP'][:extractNum].to_csv(snpsPath, sep=' ', index=False)  

    outPath = dataDir+'/biohack_'+str(extractNum)
    # --allele-1234
    plinkCmd = 'plink2/plink --file '+inputPedMap+' --allele-1234 --extract '+snpsPath+' --recode --out '+outPath

    print(plinkCmd)

    subprocess.run(plinkCmd , shell=True, check=True)



plink2/plink --file data/HACK_15_one --allele-1234 --extract data/SNPs250.txt --recode --out data/biohack_250
plink2/plink --file data/HACK_15_one --allele-1234 --extract data/SNPs500.txt --recode --out data/biohack_500
plink2/plink --file data/HACK_15_one --allele-1234 --extract data/SNPs750.txt --recode --out data/biohack_750
plink2/plink --file data/HACK_15_one --allele-1234 --extract data/SNPs1000.txt --recode --out data/biohack_1000


# Prepare Data

In [4]:

for extractNum in extractNums:
    with open(dataDir+'/biohack_'+str(extractNum)+'.ped') as file:
        with open(dataDir+'/biohack_'+str(extractNum)+'_AA_no.ped', 'w') as writeF:
            for nom in range(5901):
                line = file.readline()
                line = line.replace('I', '5')
                line = line.replace('D', '6')
                line = line.split(' ')

                if line[5] == '-9':
                    continue
                

                arr = []
                for num in range(6):
                    arr.append(line[num])


                for i in range(6,len(line), 2):
                    arr.append(line[i] + line[i+1]) 


                writeF.write(' '.join(arr))
            

# Prediction
Zastosowanie metod uczenia nadzorowanego do przewidywania koloru oczu. 

In [5]:
import pandas as pd
import numpy as np

## Prepare training and testing data

In [6]:
results = pd.DataFrame(columns=['Num_of_rs', 'algo', 'score', 'params'])
results

,Num_of_rs,algo,score,params


In [7]:
importances = {}

In [44]:
from sklearn.model_selection import train_test_split

extractNum = 750

inPath = 'data/biohack_'+str(extractNum)+'_AA_no.ped'

pedData = pd.read_csv(inPath, sep=' ', names=range(75))

x_train, x_test, y_train, y_test = train_test_split(pedData.iloc[:,5:], pedData.iloc[:,4], test_size = 0.1)


In [50]:
len(x_train)

4297

## SVM

In [45]:
from sklearn import svm

params = 'default'

clf = svm.SVC()
clf.fit(x_train, y_train)

score = clf.score(x_test, y_test)

df = pd.DataFrame([[extractNum, 'SVM', score, params]], columns=['Num_of_rs', 'algo', 'score', 'params'])
results = results.append(df, ignore_index=True)
results
# df

,Num_of_rs,algo,score,params
0,150,SVM,0.682008,default
1,150,MLP,0.895397,default
2,250,SVM,0.587866,default
3,250,MLP,0.581590,default
4,250,SGD,0.543933,default
5,250,Tree,0.479079,default
6,500,SVM,0.767782,default
7,500,MLP,0.759414,default
8,500,SGD,0.217573,default
9,500,Tree,0.684100,default


## Neural Network - MLPClassifier

In [46]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs')
clf.fit(x_train, y_train)

score = clf.score(x_test, y_test)

df = pd.DataFrame([[extractNum, 'MLP', score, params]], columns=['Num_of_rs', 'algo', 'score', 'params'])
results = results.append(df, ignore_index=True)
results

,Num_of_rs,algo,score,params
0,150,SVM,0.682008,default
1,150,MLP,0.895397,default
2,250,SVM,0.587866,default
3,250,MLP,0.581590,default
4,250,SGD,0.543933,default
5,250,Tree,0.479079,default
6,500,SVM,0.767782,default
7,500,MLP,0.759414,default
8,500,SGD,0.217573,default
9,500,Tree,0.684100,default


## Stochastic Gradient Descent

In [47]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
clf.fit(x_train, y_train)

score = clf.score(x_test, y_test)

df = pd.DataFrame([[extractNum, 'SGD', score, params]], columns=['Num_of_rs', 'algo', 'score', 'params'])
results = results.append(df, ignore_index=True)
results

,Num_of_rs,algo,score,params
0,150,SVM,0.682008,default
1,150,MLP,0.895397,default
2,250,SVM,0.587866,default
3,250,MLP,0.581590,default
4,250,SGD,0.543933,default
5,250,Tree,0.479079,default
6,500,SVM,0.767782,default
7,500,MLP,0.759414,default
8,500,SGD,0.217573,default
9,500,Tree,0.684100,default


## DecisionTreeClassifier

In [48]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)

score = clf.score(x_test, y_test)

df = pd.DataFrame([[extractNum, 'Tree', score, params]], columns=['Num_of_rs', 'algo', 'score', 'params'])
results = results.append(df, ignore_index=True)
results

# 

,Num_of_rs,algo,score,params
0,150,SVM,0.682008,default
1,150,MLP,0.895397,default
2,250,SVM,0.587866,default
3,250,MLP,0.581590,default
4,250,SGD,0.543933,default
5,250,Tree,0.479079,default
6,500,SVM,0.767782,default
7,500,MLP,0.759414,default
8,500,SGD,0.217573,default
9,500,Tree,0.684100,default


In [37]:
results.to_csv('results2.csv', sep='\t')

In [54]:
# importances = {extractNum: clf.feature_importances_}
# # importances = np.append(importances, clf.feature_importances_)
# importances

{100: array([  6.88048028e-01,   4.70573080e-02,   8.83296191e-03,
          8.65205183e-03,   9.87926986e-03,   6.09465691e-02,
          2.81823832e-03,   3.05877052e-03,   3.59636431e-03,
          2.05939608e-03,   4.26357940e-03,   1.92088463e-03,
          2.54879477e-03,   1.72539827e-03,   2.11351722e-03,
          2.27623721e-03,   3.28910728e-03,   0.00000000e+00,
          1.59641620e-03,   1.12488756e-03,   2.59592262e-03,
          6.01088090e-04,   1.52097100e-03,   1.09047127e-02,
          2.56798597e-04,   2.42305430e-03,   1.88638475e-03,
          1.27463903e-03,   2.24033575e-03,   9.58912570e-04,
          2.56857223e-03,   1.55137725e-03,   5.70981004e-03,
          1.85544855e-04,   2.15328455e-03,   2.92013028e-03,
          1.43718404e-03,   4.39088586e-03,   1.66861846e-03,
          2.95475885e-04,   3.81183833e-04,   2.39548604e-03,
          4.17108989e-03,   1.79887204e-03,   8.95722286e-04,
          3.84298060e-03,   0.00000000e+00,   1.06235498e-03,
   